This notebook explores both model index and opc ua scripts and contain examples of all the functions to make request to model index api and opc ua api servers.  

### Import Libraries

In [1]:
# Import the required packeages
import pandas as pd
from typing import List, Dict
import sys
import os
import asyncio
import datetime

### Import Scripts

In [2]:
# Setting the path is only needed if you are running this locally from source.
# Consider using pip to install instead.
module_path = os.path.abspath(os.path.join("../src/pyprediktormapclient/"))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/Users/andreaslydersen/Documents/Workspace/prediktor/pyPrediktorMapClient/src/pyprediktormapclient


In [3]:
# Import model index functions
from pyprediktormapclient.model_index import ModelIndex

# Import OPC UA functions
from pyprediktormapclient.opc_ua import OPC_UA

# Import Analytics Helper
from pyprediktormapclient.analytics_helper import AnalyticsHelper

# Import "Dataframer" Tools
from pyprediktormapclient.shared import *

In [4]:
# Connection to the servers
model_index_url = "http://10.100.59.152:13371/v1/"
opcua_rest_url = "https://apis-opcua-api.pview.dev/"
opcua_server_url = "opc.tcp://81.166.54.193:4853"

# Model index API
model = ModelIndex(url=model_index_url)

# OPC UA API
tsdata = OPC_UA(rest_url=opcua_rest_url, opcua_url=opcua_server_url)

### Download data from modelindex api

In [5]:
# Listed sites on the model index api server
namespaces = model.get_namespace_array()
namespaces

[{'Idx': 0, 'Uri': 'http://opcfoundation.org/UA/'},
 {'Idx': 1, 'Uri': 'http://prediktor.no/apis/ua/'},
 {'Idx': 2, 'Uri': 'urn:prediktor:UAA-W2022-01:Scatec'},
 {'Idx': 3, 'Uri': 'http://scatecsolar.com/EG-AS'},
 {'Idx': 4, 'Uri': 'http://scatecsolar.com/Enterprise'},
 {'Idx': 5, 'Uri': 'http://scatecsolar.com/JO-GL'},
 {'Idx': 6, 'Uri': 'http://prediktor.no/PVTypes/'}]

In [ ]:
# Types of Objects
object_types_json = model.get_object_types()
object_types = AnalyticsHelper(object_types_json)
object_types.dataframe

In [ ]:
# Unique types of Objects
object_types_unique = object_types.dataframe[["Id", "Name"]].drop_duplicates()
object_types_unique

In [ ]:
# To get typeId by type name of an object
object_type_id = model.get_object_type_id_from_name("SiteType")
object_type_id

In [6]:
# To get the objects of a type
sites_json = model.get_objects_of_type("SiteType")

# Send the returned JSON into a normalizer to get Id, Type, Name, Props and Vars as columns
sites = AnalyticsHelper(sites_json)
sites.list_of_names()

['JO-GL', 'EG-AS']

In [7]:
# Analytics helper
sites.variables_as_dataframe()

,Id,Type,Name,VariableId,VariableName,VariableIdSplit
0,3:1:SSO.EG-AS,6:0:1009,EG-AS,3:1:SSO.EG-AS.Signals.Alarm.CommLossPlantDevice,Alarm.CommLossPlantDevice,{'Id': 'SSO.EG-AS.Signals.Alarm.CommLossPlantD...
0,3:1:SSO.EG-AS,6:0:1009,EG-AS,3:1:SSO.EG-AS.Signals.State.IsDay,State.IsDay,"{'Id': 'SSO.EG-AS.Signals.State.IsDay', 'Names..."
0,3:1:SSO.EG-AS,6:0:1009,EG-AS,3:1:SSO.EG-AS.Signals.State.CommLossPlantDevic...,State.CommLossPlantDeviceCount,{'Id': 'SSO.EG-AS.Signals.State.CommLossPlantD...
0,3:1:SSO.EG-AS,6:0:1009,EG-AS,3:1:SSO.EG-AS.Signals.State.TotalPlantDeviceCount,State.TotalPlantDeviceCount,{'Id': 'SSO.EG-AS.Signals.State.TotalPlantDevi...
0,3:1:SSO.EG-AS,6:0:1009,EG-AS,3:1:SSO.EG-AS.Signals.Status,Status,"{'Id': 'SSO.EG-AS.Signals.Status', 'Namespace'..."
...,...,...,...,...,...,...
1,5:1:SSO.JO-GL,6:0:1009,JO-GL,5:1:SSO.JO-GL.Signals.Weather.SoilingIndex,Weather.SoilingIndex,{'Id': 'SSO.JO-GL.Signals.Weather.SoilingIndex...
1,5:1:SSO.JO-GL,6:0:1009,JO-GL,5:1:SSO.JO-GL.Signals.Weather.WindDirection,Weather.WindDirection,{'Id': 'SSO.JO-GL.Signals.Weather.WindDirectio...
1,5:1:SSO.JO-GL,6:0:1009,JO-GL,5:1:SSO.JO-GL.Signals.Weather.WindSpeed,Weather.WindSpeed,"{'Id': 'SSO.JO-GL.Signals.Weather.WindSpeed', ..."
1,5:1:SSO.JO-GL,6:0:1009,JO-GL,5:1:SSO.JO-GL.Signals.ACActivePower,ACActivePower,"{'Id': 'SSO.JO-GL.Signals.ACActivePower', 'Nam..."


In [ ]:
sites.list_of_ids()

In [ ]:
# String sets for all parks
string_sets_json = model.get_object_descendants(
    "StringSetType", sites.list_of_ids(), "PV_Assets"
)
string_sets = AnalyticsHelper(string_sets_json).dataframe
string_sets

In [9]:
# Selecting the first site
first_site_id = sites.list_of_ids()[0]

In [10]:
# Get all stringsets for one park
string_sets_for_first_park_as_json = model.get_object_descendants(
    "StringSetType", [first_site_id], "PV_Assets"
)
string_sets_for_first_park = AnalyticsHelper(string_sets_for_first_park_as_json)
string_sets_for_first_park.dataframe

,Id,Name,Type,Props,Vars
0,5:1:SSO.JO-GL.S1.Z2.TS01.I01.SM01.CH01,JO-GL-TS01-I01-SM01-CH01,StringSetType,"[{'DisplayName': 'ColPosition', 'Value': '1'},...","[{'DisplayName': 'Alarm.StringDisconnected', '..."
1,5:1:SSO.JO-GL.S1.Z2.TS01.I01.SM01.CH02,JO-GL-TS01-I01-SM01-CH02,StringSetType,"[{'DisplayName': 'ColPosition', 'Value': '1'},...","[{'DisplayName': 'Alarm.StringDisconnected', '..."
2,5:1:SSO.JO-GL.S1.Z2.TS01.I01.SM01.CH03,JO-GL-TS01-I01-SM01-CH03,StringSetType,"[{'DisplayName': 'ColPosition', 'Value': '1'},...","[{'DisplayName': 'Alarm.StringDisconnected', '..."
3,5:1:SSO.JO-GL.S1.Z2.TS01.I01.SM01.CH04,JO-GL-TS01-I01-SM01-CH04,StringSetType,"[{'DisplayName': 'ColPosition', 'Value': '1'},...","[{'DisplayName': 'Alarm.StringDisconnected', '..."
4,5:1:SSO.JO-GL.S1.Z2.TS01.I01.SM01.CH05,JO-GL-TS01-I01-SM01-CH05,StringSetType,"[{'DisplayName': 'ColPosition', 'Value': '1'},...","[{'DisplayName': 'Alarm.StringDisconnected', '..."
...,...,...,...,...,...
897,5:1:SSO.JO-GL.S1.Z1.TS06.I12.SM07.CH02,JO-GL-TS06-I12-SM07-CH02,StringSetType,"[{'DisplayName': 'ColPosition', 'Value': '33'}...","[{'DisplayName': 'Alarm.StringDisconnected', '..."
898,5:1:SSO.JO-GL.S1.Z1.TS06.I12.SM07.CH03,JO-GL-TS06-I12-SM07-CH03,StringSetType,"[{'DisplayName': 'ColPosition', 'Value': '34'}...","[{'DisplayName': 'Alarm.StringDisconnected', '..."
899,5:1:SSO.JO-GL.S1.Z1.TS06.I12.SM07.CH04,JO-GL-TS06-I12-SM07-CH04,StringSetType,"[{'DisplayName': 'ColPosition', 'Value': '34'}...","[{'DisplayName': 'Alarm.StringDisconnected', '..."
900,5:1:SSO.JO-GL.S1.Z1.TS06.I12.SM07.CH05,JO-GL-TS06-I12-SM07-CH05,StringSetType,"[{'DisplayName': 'ColPosition', 'Value': '34'}...","[{'DisplayName': 'Alarm.StringDisconnected', '..."


In [11]:
# Ancestors of an object type, get all trackers that are ancestor of the parks string sets

trackers_as_json = model.get_object_ancestors(
    "TrackerType", string_sets_for_first_park.list_of_ids(), "PV_Serves"
)
trackers = AnalyticsHelper(trackers_as_json)
trackers.variables_as_dataframe()

,Id,Name,Type,VariableId,VariableName,VariableIdSplit
0,5:1:SSO.JO-GL.S1.MMS1.1.TS01-TR1.TC_1.1.1,JO-GL-TS01-TR1.TC_1.1.1,TrackerType,5:1:SSO.JO-GL.S1.MMS1.1.TS01-TR1.TC_1.1.1.Sign...,AngleMeasured,{'Id': 'SSO.JO-GL.S1.MMS1.1.TS01-TR1.TC_1.1.1....
0,5:1:SSO.JO-GL.S1.MMS1.1.TS01-TR1.TC_1.1.1,JO-GL-TS01-TR1.TC_1.1.1,TrackerType,5:1:SSO.JO-GL.S1.MMS1.1.TS01-TR1.TC_1.1.1.Sign...,AngleSetpoint,{'Id': 'SSO.JO-GL.S1.MMS1.1.TS01-TR1.TC_1.1.1....
0,5:1:SSO.JO-GL.S1.MMS1.1.TS01-TR1.TC_1.1.1,JO-GL-TS01-TR1.TC_1.1.1,TrackerType,5:1:SSO.JO-GL.S1.MMS1.1.TS01-TR1.TC_1.1.1.Para...,TrackingLimitEastAngle,{'Id': 'SSO.JO-GL.S1.MMS1.1.TS01-TR1.TC_1.1.1....
0,5:1:SSO.JO-GL.S1.MMS1.1.TS01-TR1.TC_1.1.1,JO-GL-TS01-TR1.TC_1.1.1,TrackerType,5:1:SSO.JO-GL.S1.MMS1.1.TS01-TR1.TC_1.1.1.Para...,TrackingLimitWestAngle,{'Id': 'SSO.JO-GL.S1.MMS1.1.TS01-TR1.TC_1.1.1....
0,5:1:SSO.JO-GL.S1.MMS1.1.TS01-TR1.TC_1.1.1,JO-GL-TS01-TR1.TC_1.1.1,TrackerType,5:1:SSO.JO-GL.S1.MMS1.1.TS01-TR1.TC_1.1.1.Sign...,Status,{'Id': 'SSO.JO-GL.S1.MMS1.1.TS01-TR1.TC_1.1.1....
...,...,...,...,...,...,...
901,5:1:SSO.JO-GL.S1.MMS5.6.TS03-TR2.TC_5.6.3,JO-GL-TS03-TR2.TC_5.6.3,TrackerType,5:1:SSO.JO-GL.S1.MMS5.6.TS03-TR2.TC_5.6.3.Sign...,AngleMeasured,{'Id': 'SSO.JO-GL.S1.MMS5.6.TS03-TR2.TC_5.6.3....
901,5:1:SSO.JO-GL.S1.MMS5.6.TS03-TR2.TC_5.6.3,JO-GL-TS03-TR2.TC_5.6.3,TrackerType,5:1:SSO.JO-GL.S1.MMS5.6.TS03-TR2.TC_5.6.3.Sign...,AngleSetpoint,{'Id': 'SSO.JO-GL.S1.MMS5.6.TS03-TR2.TC_5.6.3....
901,5:1:SSO.JO-GL.S1.MMS5.6.TS03-TR2.TC_5.6.3,JO-GL-TS03-TR2.TC_5.6.3,TrackerType,5:1:SSO.JO-GL.S1.MMS5.6.TS03-TR2.TC_5.6.3.Para...,TrackingLimitEastAngle,{'Id': 'SSO.JO-GL.S1.MMS5.6.TS03-TR2.TC_5.6.3....
901,5:1:SSO.JO-GL.S1.MMS5.6.TS03-TR2.TC_5.6.3,JO-GL-TS03-TR2.TC_5.6.3,TrackerType,5:1:SSO.JO-GL.S1.MMS5.6.TS03-TR2.TC_5.6.3.Para...,TrackingLimitWestAngle,{'Id': 'SSO.JO-GL.S1.MMS5.6.TS03-TR2.TC_5.6.3....


### Download data from the opc ua api

In [12]:
# Live value data of trackers
live_value = tsdata.get_values(
    trackers.variables_as_list(["AngleMeasured", "AngleSetpoint"])
)
live_value

[{'Id': 'SSO.JO-GL.S1.MMS1.1.TS01-TR1.TC_1.1.1.Signals.AngleMeasured',
  'Namespace': 5,
  'IdType': 1,
  'Timestamp': '2022-09-29T12:35:44.3940759Z',
  'Value': 45.2,
  'ValueType': 'Float',
  'StatusCode': None,
  'StatusSymbol': None},
 {'Id': 'SSO.JO-GL.S1.MMS1.1.TS01-TR1.TC_1.1.1.Signals.AngleSetpoint',
  'Namespace': 5,
  'IdType': 1,
  'Timestamp': '2022-09-29T12:35:44.3929622Z',
  'Value': 45,
  'ValueType': 'Float',
  'StatusCode': None,
  'StatusSymbol': None},
 {'Id': 'SSO.JO-GL.S1.MMS1.1.TS01-TR1.TC_1.1.1.Signals.AngleMeasured',
  'Namespace': 5,
  'IdType': 1,
  'Timestamp': '2022-09-29T12:35:44.3940759Z',
  'Value': 45.2,
  'ValueType': 'Float',
  'StatusCode': None,
  'StatusSymbol': None},
 {'Id': 'SSO.JO-GL.S1.MMS1.1.TS01-TR1.TC_1.1.1.Signals.AngleSetpoint',
  'Namespace': 5,
  'IdType': 1,
  'Timestamp': '2022-09-29T12:35:44.3929622Z',
  'Value': 45,
  'ValueType': 'Float',
  'StatusCode': None,
  'StatusSymbol': None},
 {'Id': 'SSO.JO-GL.S1.MMS1.1.TS01-TR1.TC_1.1.1.S

In [ ]:
# Input parameters for value data
# Parameters for aggregate historical data
start_time = (datetime.datetime.now() - datetime.timedelta(30)).strftime(
    "%Y-%m-%dT%H:%M:%SZ"
)  # 30 days ago
end_time = (datetime.datetime.now() - datetime.timedelta(29)).strftime(
    "%Y-%m-%dT%H:%M:%SZ"
)  # 29 days ago, so one day of data
pro_interval = 3600000  # 10 minutes processing time
agg_name = "Average"

In [20]:
tsdata.get_historical_aggregated_values(
    start_time=(datetime.datetime.now() - datetime.timedelta(30)),
    end_time=(datetime.datetime.now() - datetime.timedelta(29)),
    pro_interval=3600000,
    agg_name="Average",
    variable_list=trackers.variables_as_list(["AngleMeasured", "AngleSetpoint"]),
)

In [ ]:
# 1 day aggregated historical trackers data
await tsdata.get_agg_hist_value_data(
    start_time,
    end_time,
    pro_interval,
    agg_name,
    trackers.dataframe,
    ["AngleMeasured", "AngleSetpoint"],
)

In [ ]:
# Reading downloaded tracker data
tracker_df = pd.read_parquet("Data/data_chunk_0.parquet")
tracker_df